# 절차
- 시퀀스 -> add() -> compile() -> fit -> 평가

In [0]:
import keras
from keras import models, layers, backend
from keras.datasets import mnist
%tensorflow_version 1.x

## 데이터 확보/준비

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [33]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape
# y_train, y_test는 벡터화 처리 해야 함

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [0]:
# 이미지 크기
img_rows, img_cols = x_train.shape[1:]

In [35]:
# 채널을 설명하는 값에 의해 데이터를 보정
# Demension 확장(keras에만 있는 부분)
backend.image_data_format()

'channels_last'

In [0]:
# 입력 데이터 보정(reshape)
if backend.image_data_format() == 'channels_first':
  # [60000, 28, 28] -> [60000, 1, 28, 28]
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  # [60000, 28, 28] -> [60000, 28, 28, 1]
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
# 데이터의 정규화: ~/256(총 분류 개수로 정규화), ~/255(최대값으로 정규화)
x_train /= 255
x_test /= 255

In [39]:
# 레이블 벡터화
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [0]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [41]:
y_train.shape, y_train[0]

((60000, 10), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

## CNN 레이어 설계

In [42]:
# 모델준비(시퀀스)
model = models.Sequential()

In [44]:
# 합성곱층 추가: 채널 수 증가, 특징 추출
model.add(layers.Conv2D(filters=32,        # 출력채널수
                        kernel_size=(3,3), # 가중치 shape
                        strides=(1,1),     # 커널 이동 칸 default:(1,1)
                        padding='valid',    # 보정, valid: 유효한 만큼만 행렬 늘림
                        activation='relu',  # 활성화 함수
                        input_shape=input_shape))

In [46]:
# 풀링층 추가: 이미지 사이즈 감소, 특징 강화
model.add(layers.MaxPool2D(pool_size=(2,2),
                           strides=None # 기본값 적용
                          ))

In [47]:
# 과적합 방지: 학습 방해
model.add(layers.Dropout(0.25)) # 임의의 값


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# 합성곱층 추가: 채널 수 증가, 특징 추출
model.add(layers.Conv2D(filters=64, kernel_size=(3,3),activation='relu'))

In [0]:
# 풀링층 추가: 이미지 사이즈 감소, 특징 강화
model.add(layers.MaxPool2D(pool_size=(2,2)))

In [0]:
# 과적합 방지: 학습 방해
model.add(layers.Dropout(0.25))

In [0]:
# 데이터를 펴줌
model.add(layers.Flatten())

In [0]:
# 전결합층
# layers.Dense: 일반적 계층 1개를 의미
# 7 x 7 x 64 => Flatten => 128에 수렴
model.add(layers.Dense(128, activation='relu'))

In [0]:
# 과적합 방지: 학습 방해
model.add(layers.Dropout(0.5))

In [0]:
# 출력층: 데이터를 레이블의 분류 개수만큼 수렴
model.add(layers.Dense(10, activation='softmax'))

## 학습 및 평가

In [56]:
# 크로스 엔트로피, 경사 하강법
model.compile(loss = keras.losses.categorical_crossentropy, 
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [0]:
epochs = 10       # 훈련 세대 수(=총 학습의 횟수)
batch_size = 128  # 1회 학습 데이터 양

In [58]:
model.fit(x_train, y_train ,batch_size, epochs, validation_split=0.25)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 45000 samples, validate on 15000 samples
Epoch 1/10





45000/45000 [==============================] - 43s 950us/step - loss: 0.3844 - acc: 0.8796 - val_loss: 0.0926 - val_acc: 0.9713
Epoch 2/10
45000/45000 [==============================] - 42s 933us/step - loss: 0.1260 - acc: 0.9612 - val_loss: 0.0607 - val_acc: 0.9819
Epoch 3/10
45000/45000 [==============================] - 42s 936us/step - loss: 0.0917 - acc: 0.9730 - val_loss: 0.0506 - val_acc: 0.9837
Epoch 4/10
45000/45000 [==============================] - 42s 931us/step - loss: 0.0759 - acc: 0.9768 - val_loss: 0.0444 - val_acc: 0.9867
Epoch 5/10
45000/45000 [==============================] - 42s 938us/step - loss: 0.0693 - acc: 0.9794 - val_loss: 0.0425 - val_acc: 0.9874
Epoch 6/10
45000/45000 [==============================] - 42s 932us/step - loss: 0.0625 - acc: 0.9816 - val_loss: 0.0404 - val_acc: 0.9884
Epoch 7/10
450

In [59]:
# 점수
score = model.evaluate(x_test, y_test)
score

10000/10000 [==============================] - 3s 311us/step


[0.027430314925371204, 0.9908]